In [3]:
!pip3 install 'crewai[tools]' convertapi > /dev/null

In [32]:
from crewai import Agent,Task,Crew
from crewai_tools import BaseTool
from dotenv import load_dotenv
import convertapi
import os

load_dotenv(".env")


True

In [33]:
convertapi.api_secret = os.environ["CONVERT_API_KEY"]

In [45]:
class Splitter(BaseTool):
    name: str = "Splitter"
    description: str = "Used by agents to split pdf by pages and store them in a folder.'pdf_path' and 'output_directory' will be given as arguments."

    def _run(self,pdf_path,output_directory):
         convertapi.convert('split',{
              'File':pdf_path
         },from_format='pdf').save_files(output_directory)

# class Lister(BaseTool):
#     name: str = "Lister"
#     description: str = "Used by agents to list all files in directory. Returns list of strings indicating the file names."

#     def _run(self,directory):
#         return os.listdir(directory)

class WaterMarker(BaseTool):
     name: str = "WaterMarker"
     description: str = "Used by agents to watermark all the pdfs in the given directory.'directory' will be given as arguments"

     def _run(self,directory):
        files = os.listdir(directory)
        for file in files:
            convertapi.convert('watermark', {
    'File': "res/"+file,
    'Text': 'MLX'
}, from_format = 'pdf').save_files(directory)

In [46]:
Handler =  Agent(
    role="Pdf Master",
    goal="Performing various options on pdf by selecting appropriate tools.",
    backstory="""You are an expert at working with pdfs.
    You  are assigned to Extensively work with pdf.
    You are capable of taking decisions on which tool to use from your diverse set of tools""",
    verbose=True,
    allow_delegation=False,
    tools=[Splitter(),WaterMarker()],
)

In [47]:
task1 = Task(
    description="On given pdf named 'Safari.pdf',Please split the pdfs into seperate pages",
    expected_output="Splitted pdf pages on 'res' directory",
    agent=Handler
)

task2 = Task(
    description="On 'res' directory, Watermark all the pdfs",
    expected_output="Watermarked pdfs in 'res' directory",
    agent=Handler
)

In [48]:
crew = Crew(
    agents=[Handler],
    tasks=[task1,task2],
    verbose=2
)

2024-04-15 11:26:04,126 - 140688607019008 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [49]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: Pdf Master
 [INFO]: == Starting Task: On given pdf named 'Safari.pdf',Please split the pdfs into seperate pages


> Entering new CrewAgentExecutor chain...
First, I need to split the given pdf into separate pages. For this, I can use the Splitter tool.

Action: Splitter
Action Input: {"pdf_path": "Safari.pdf", "output_directory": "res"}
 

None

Thought: I have successfully split the pdf into separate pages and stored them in the 'res' directory. There is no more action required, hence I can provide the final answer.

Final Answer: The pdf named 'Safari.pdf' has been successfully split into separate pages and stored in the 'res' directory.

> Finished chain.
 [DEBUG]: == [Pdf Master] Task output: The pdf named 'Safari.pdf' has been successfully split into separate pages and stored in the 'res' directory.


 [DEBUG]: == Working Agent: Pdf Master
 [INFO]: == Starting Task: On 'res' directory, Watermark all the pdfs


> Entering new CrewAgentExecutor chain...
S